# PHASE 6 — RAG Pipeline Integration

**Objectives:**
- Build knowledge base from PHASE 5 degradation data
- Implement document chunking and embedding
- Create FAISS vector store for similarity search
- Retrieve similar historical failures with citations
- Validate retrieval relevance

**Expected Outcomes:**
- Knowledge base with 100-300 failure incidents
- Similarity search with >70% top-5 recall
- Citation tracking for all retrieved incidents
- Query: "Find past incidents similar to current sensor deviation pattern"

## Section 1: Setup & Imports

Import all RAG modules and configure logging.

In [1]:
# ══════════════════════════════════════════════════════════════════════
# MUST be first — set thread / parallelism env vars BEFORE any import
# (prevents OpenMP / MKL / tokenizers fork-deadlock in Jupyter kernels)
# ══════════════════════════════════════════════════════════════════════
import os
os.environ['TOKENIZERS_PARALLELISM'] = 'false'
os.environ['HF_HUB_OFFLINE']        = '1'
os.environ['TRANSFORMERS_OFFLINE']   = '1'
os.environ['OMP_NUM_THREADS']        = '1'
os.environ['MKL_NUM_THREADS']        = '1'
os.environ['OPENBLAS_NUM_THREADS']   = '1'
os.environ['VECLIB_MAXIMUM_THREADS'] = '1'   # macOS Accelerate
os.environ['NUMEXPR_MAX_THREADS']    = '1'

# Standard library
import importlib
import logging
import sys
import warnings
from pathlib import Path

# ── Ensure project root is on sys.path ──────────────────────────────
project_root = Path.cwd().parent
if str(project_root) not in sys.path:
    sys.path.insert(0, str(project_root))

# Data science
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# Limit PyTorch threads (belt-and-suspenders after env vars)
import torch
torch.set_num_threads(1)
torch.set_num_interop_threads(1)

# ── Force-reload edited source modules so changes take effect ────────
import src.features.sliding_windows as _sw_mod;   importlib.reload(_sw_mod)
import src.features.feature_selection as _fs_mod;  importlib.reload(_fs_mod)
import src.features.pipeline as _fp_mod;           importlib.reload(_fp_mod)
import src.rag.knowledge_base as _kb_mod;          importlib.reload(_kb_mod)
import src.rag;                                    importlib.reload(src.rag)

# RAG pipeline
from src.rag import (
    DocumentChunker,
    Embedder,
    VectorStore,
    Retriever,
    KnowledgeBase
)
from src.rag.knowledge_base import create_test_cases_from_degradation

# Previous phases — corrected import paths
from src.ingestion.cmapss_loader import CMAPSSDataLoader
from src.features.pipeline import FeatureEngineeringPipeline
from src.models.baseline_ml import XGBoostRULPredictor
from src.anomaly import (
    ResidualAnomalyDetector,
    ChangePointDetector,
    DegradationLabeler,
    EarlyWarningSystem
)

# Configure logging
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(name)s - %(levelname)s - %(message)s'
)
logger = logging.getLogger(__name__)

# Suppress chatty HTTP / tokenizer / RAG module logs
for _lg in ['httpx', 'sentence_transformers', 'transformers',
            'src.rag', 'src.rag.knowledge_base', 'src.rag.document_chunker',
            'src.rag.embedder', 'src.rag.vector_store', 'src.rag.retriever']:
    logging.getLogger(_lg).setLevel(logging.WARNING)

# Suppress warnings
warnings.filterwarnings('ignore')

# Set style
sns.set_style('whitegrid')
plt.rcParams['figure.figsize'] = (14, 8)
plt.rcParams['font.size'] = 10

# Constants
SUBSETS = ['FD001', 'FD002', 'FD003', 'FD004']
SENSOR_COLS = [f'sensor_{i}' for i in range(1, 22)]
DATA_DIR = Path('../data/raw/CMAPSS')
REPORTS_DIR = Path('../reports/figures')
REPORTS_DIR.mkdir(parents=True, exist_ok=True)
KB_DIR = Path('../data/vector_db')
KB_DIR.mkdir(parents=True, exist_ok=True)

print("✓ All modules imported successfully (source reloaded)")
print(f"✓ torch.get_num_threads() = {torch.get_num_threads()}")
print(f"✓ Subsets: {SUBSETS}")
print(f"✓ Notebook ready for PHASE 6: RAG Pipeline Integration")

2026-02-27 02:33:22 - root - INFO - Logging configured. Level: INFO, File: logs/ewis.log


✓ All modules imported successfully (source reloaded)
✓ torch.get_num_threads() = 1
✓ Subsets: ['FD001', 'FD002', 'FD003', 'FD004']
✓ Notebook ready for PHASE 6: RAG Pipeline Integration


## Section 2: Generate Degradation Data (PHASE 5 Output)

Run PHASE 5 pipeline to generate degradation periods for knowledge base.

In [2]:
# ── 2A. Load ALL 4 C-MAPSS subsets ──────────────────────────────────
logger.info("Loading all 4 C-MAPSS subsets...")
loader = CMAPSSDataLoader(data_dir=str(DATA_DIR))

train_frames, test_frames = [], []
for subset in SUBSETS:
    tr, te, rul_te = loader.load_dataset(subset)
    # Composite engine IDs to prevent cross-subset contamination
    tr = tr.copy()
    te = te.copy()
    tr['engine_id'] = subset + '_' + tr['engine_id'].astype(int).astype(str)
    te['engine_id'] = subset + '_' + te['engine_id'].astype(int).astype(str)
    # Assign RUL to test rows (last-cycle RUL broadcast)
    max_cycles = te.groupby('engine_id')['cycle'].transform('max')
    te['RUL'] = (max_cycles - te['cycle']).values + rul_te.values[
        te.groupby('engine_id').ngroup().values
    ]
    train_frames.append(tr)
    test_frames.append(te)
    print(f"  {subset}: train {len(tr):,} rows / {tr['engine_id'].nunique()} engines  |  "
          f"test {len(te):,} rows / {te['engine_id'].nunique()} engines")

df_train = pd.concat(train_frames, ignore_index=True)
df_test  = pd.concat(test_frames,  ignore_index=True)
print(f"\n✓ Combined train: {len(df_train):,} rows, {df_train['engine_id'].nunique()} engines")
print(f"✓ Combined test:  {len(df_test):,} rows, {df_test['engine_id'].nunique()} engines")

# ── 2B. Feature engineering ─────────────────────────────────────────
logger.info("Engineering features...")
pipeline = FeatureEngineeringPipeline(window_size=30, window_step=1, scale_features=True)
X_train, y_train = pipeline.fit_transform(
    df_train, sensor_cols=SENSOR_COLS, target_col='RUL',
    feature_selection_method='combined', selection_k=20
)
X_test, y_test = pipeline.transform(df_test)
print(f"✓ Features — train {X_train.shape}, test {X_test.shape}")

# ── 2C. Train XGBoost model ────────────────────────────────────────
logger.info("Training XGBoost model...")
model = XGBoostRULPredictor()
model.fit(X_train, y_train, X_test[:1000], y_test[:1000])

y_pred = model.predict(X_test)
residuals = y_test - y_pred

# ── 2D. Anomaly detection pipeline ─────────────────────────────────
logger.info("Detecting anomalies...")
residual_detector = ResidualAnomalyDetector(method='zscore', threshold=3.0)
residual_detector.fit(residuals[:1000])
anomalies = residual_detector.detect(residuals)
anomaly_scores = residual_detector.get_anomaly_scores(residuals)

logger.info("Detecting change points...")
cp_detector = ChangePointDetector(method='cusum', threshold=3.0)
cp_detector.fit(y_test[:100])
change_points = cp_detector.detect(y_test)

# ── 2E. Degradation labeling ───────────────────────────────────────
logger.info("Labeling degradation periods...")
labeler = DegradationLabeler(rul_threshold=100)
degradation_df = labeler.label_degradation(
    rul_values=y_test,
    anomaly_flags=anomalies,
    anomaly_scores=anomaly_scores,
    change_points=change_points
)
degradation_periods = labeler.get_degradation_periods(degradation_df)

# ── 2F. Normalise period keys & map engine IDs ─────────────────────
# get_degradation_periods() returns 'start_idx'/'end_idx';
# build_from_degradation_data() expects 'start'/'end'.
engine_ids = df_test['engine_id'].values
for period in degradation_periods:
    # Add canonical 'start'/'end' keys expected by KnowledgeBase
    period['start'] = period['start_idx']
    period['end']   = period['end_idx']
    # Map composite engine ID from the test-set index
    idx = period['start_idx']
    if idx < len(engine_ids):
        period['engine_id'] = str(engine_ids[idx])

# Per-subset summary
from collections import Counter
subset_counts = Counter(
    str(p.get('engine_id', '')).split('_')[0] for p in degradation_periods
)
print(f"\n✓ Generated {len(degradation_periods)} degradation periods")
print(f"✓ Total test cycles: {len(y_test):,}")
print(f"✓ Anomalies detected: {int(np.sum(anomalies)):,} ({np.mean(anomalies):.1%})")
print(f"✓ Change points: {len(change_points)}")
print(f"✓ Per-subset periods: {dict(subset_counts)}")
if degradation_periods:
    print(f"\nExample period:\n{degradation_periods[0]}")

2026-02-27 02:33:26 - __main__ - INFO - Loading all 4 C-MAPSS subsets...
2026-02-27 02:33:26 - src.ingestion.cmapss_loader - INFO - Loaded FD001: Train=20631 rows, Test=13096 rows
2026-02-27 02:33:26 - src.ingestion.cmapss_loader - INFO - Loaded FD002: Train=53759 rows, Test=33991 rows


  FD001: train 20,631 rows / 100 engines  |  test 13,096 rows / 100 engines
  FD002: train 53,759 rows / 260 engines  |  test 33,991 rows / 259 engines


2026-02-27 02:33:26 - src.ingestion.cmapss_loader - INFO - Loaded FD003: Train=24720 rows, Test=16596 rows
2026-02-27 02:33:26 - src.ingestion.cmapss_loader - INFO - Loaded FD004: Train=61249 rows, Test=41214 rows


  FD003: train 24,720 rows / 100 engines  |  test 16,596 rows / 100 engines
  FD004: train 61,249 rows / 249 engines  |  test 41,214 rows / 248 engines


2026-02-27 02:33:26 - __main__ - INFO - Engineering features...
2026-02-27 02:33:26 - src.features.pipeline - INFO - Fitting feature engineering pipeline...
2026-02-27 02:33:26 - src.features.pipeline - INFO - Step 1: Generating sliding windows
2026-02-27 02:33:26 - src.features.sliding_windows - INFO - Generating sliding windows (size=30, step=1)



✓ Combined train: 160,359 rows, 709 engines
✓ Combined test:  104,897 rows, 707 engines


2026-02-27 02:33:30 - src.features.sliding_windows - INFO - Generated 157523 windows from 709 engines
2026-02-27 02:33:30 - src.features.sliding_windows - INFO - Window shape: (157523, 30, 24) (num_windows, window_size, features)
2026-02-27 02:33:30 - src.features.pipeline - INFO - Step 2: Calculating health indicators
2026-02-27 02:33:32 - src.features.health_indicators - INFO - Calculated combined health index using mean method
2026-02-27 02:33:32 - src.features.pipeline - INFO - Step 3: Engineering time-series features
2026-02-27 02:33:39 - src.features.engineering - INFO - Added rolling statistics for windows: [5, 10, 20]
2026-02-27 02:33:41 - src.features.engineering - INFO - Added EWMA features for spans: [5, 10, 20]
2026-02-27 02:33:41 - src.features.engineering - INFO - Added difference features for lags: [1, 5, 10]
2026-02-27 02:33:42 - src.features.engineering - INFO - Added 5 Fourier feature pairs
2026-02-27 02:33:42 - src.features.pipeline - INFO - Engineered features: 431 

KeyboardInterrupt: 

## Section 3: Build Knowledge Base

Create knowledge base from degradation periods using RAG pipeline.

In [ ]:
import time, json, subprocess, tempfile

# ── Silence verbose loggers that flood Jupyter output ────────────────
for _name in ['src.rag', 'src.rag.knowledge_base', 'src.rag.document_chunker',
              'src.rag.embedder', 'src.rag.vector_store', 'src.rag.retriever']:
    logging.getLogger(_name).setLevel(logging.WARNING)

# ═══════════════════════════════════════════════════════════════════════
# Subprocess-safe encoder
#   model.encode() deadlocks inside Jupyter kernels due to
#   tokenizers / OpenMP fork issues.  We shell out to a fresh
#   Python process that encodes and saves a .npy file.
# ═══════════════════════════════════════════════════════════════════════
def _subprocess_encode(texts, model_name='all-MiniLM-L6-v2',
                       batch_size=64, normalize=True):
    """Encode texts via subprocess to avoid Jupyter deadlock."""
    tmp_dir = tempfile.mkdtemp()
    texts_path = os.path.join(tmp_dir, 'texts.json')
    emb_path   = os.path.join(tmp_dir, 'embeddings.npy')

    with open(texts_path, 'w') as f:
        json.dump(texts if isinstance(texts, list) else [texts], f)

    script = (
        "import os, json, numpy as np;"
        "os.environ['TOKENIZERS_PARALLELISM']='false';"
        "os.environ['OMP_NUM_THREADS']='1';"
        "os.environ['MKL_NUM_THREADS']='1';"
        "from sentence_transformers import SentenceTransformer;"
        f"m=SentenceTransformer({model_name!r});"
        f"ts=json.load(open({texts_path!r}));"
        f"e=m.encode(ts,show_progress_bar=False,batch_size={batch_size},"
        f"normalize_embeddings={normalize});"
        f"np.save({emb_path!r},e)"
    )

    result = subprocess.run(
        [sys.executable, '-c', script],
        capture_output=True, text=True, timeout=300,
        env={**os.environ, 'TOKENIZERS_PARALLELISM': 'false',
             'OMP_NUM_THREADS': '1', 'MKL_NUM_THREADS': '1'}
    )
    if result.returncode != 0:
        raise RuntimeError(f"Encoding failed: {result.stderr[:500]}")

    embs = np.load(emb_path)
    os.remove(texts_path); os.remove(emb_path); os.rmdir(tmp_dir)
    return embs


# ── Step 1: Initialize Knowledge Base ────────────────────────────────
t0 = time.time()
print("Step 1/5: Loading embedding model …")
kb = KnowledgeBase(
    embedding_model='all-MiniLM-L6-v2',
    chunk_size=500,
    chunk_strategy='sentence'
)
t1 = time.time()
print(f"  done [{t1 - t0:.1f}s]")

# ── Step 2: Create documents from degradation periods ────────────────
print("Step 2/5: Creating documents …")
from src.rag.document_chunker import create_failure_document
kb.documents = []
for i, period in enumerate(degradation_periods):
    engine_id = period.get('engine_id', 0)
    sensor_stats = kb._extract_sensor_stats(df_test, engine_id,
                                             period.get('start', 0),
                                             period.get('end', 0))
    doc = create_failure_document(engine_id=engine_id,
                                  degradation_period=period,
                                  sensor_data=sensor_stats)
    kb.documents.append(doc)
t2 = time.time()
print(f"  {len(kb.documents)} documents [{t2 - t1:.1f}s]")

# ── Step 3: Chunk documents ──────────────────────────────────────────
print("Step 3/5: Chunking documents …")
kb.chunks = kb.chunker.chunk_documents(
    kb.documents, text_field='text',
    metadata_fields=['engine_id', 'failure_type', 'duration', 'severity']
)
t3 = time.time()
print(f"  {len(kb.chunks)} chunks [{t3 - t2:.1f}s]")

# ── Step 4: Generate embeddings (subprocess) ─────────────────────────
print("Step 4/5: Embedding chunks (subprocess) …")
texts = [c['text'] for c in kb.chunks]
kb.embeddings = _subprocess_encode(
    texts, model_name='all-MiniLM-L6-v2', batch_size=64,
    normalize=kb.embedder.normalize
)
t4 = time.time()
print(f"  shape {kb.embeddings.shape} [{t4 - t3:.1f}s]")

# ── Step 5: Build FAISS index & retriever, save ──────────────────────
print("Step 5/5: Building vector store & saving …")
kb.vector_store = VectorStore(
    embedding_dim=kb.embedder.embedding_dim,
    index_type='Flat', metric='cosine'
)
kb.vector_store.add(embeddings=kb.embeddings, documents=kb.chunks)
kb.retriever = Retriever(
    vector_store=kb.vector_store, embedder=kb.embedder,
    top_k=5, min_similarity=0.3, include_citations=True
)
kb.save(str(KB_DIR))
t5 = time.time()
print(f"  saved to '{KB_DIR}' [{t5 - t4:.1f}s]")

# ── Monkey-patch embedder so downstream kb.search() won't deadlock ───
_orig_embed_text = kb.embedder.embed_text

def _safe_embed_text(text, show_progress=False, batch_size=32):
    """Subprocess-safe embed_text replacement for Jupyter."""
    is_single = isinstance(text, str)
    texts_list = [text] if is_single else list(text)
    embs = _subprocess_encode(
        texts_list, model_name=kb.embedder.model_name,
        batch_size=batch_size, normalize=kb.embedder.normalize
    )
    return embs[0] if is_single else embs

kb.embedder.embed_text = _safe_embed_text
print("  ✓ Embedder patched for subprocess-safe search")

# ── Statistics ───────────────────────────────────────────────────────
stats = kb.get_statistics()
print(f"\n✓ Knowledge Base Built Successfully!")
print(f"  - Documents: {stats['n_documents']}")
print(f"  - Chunks: {stats['n_chunks']}")
print(f"  - Embedding model: {stats['embedding_model']}")
print(f"  - Embedding dimension: {stats['embedding_dim']}")
if 'mean_chunk_size' in stats:
    print(f"  - Mean chunk size: {stats['mean_chunk_size']:.0f} characters")
if 'vector_store_size' in stats:
    print(f"  - Vector store size: {stats['vector_store_size']}")

# ── Chunk size distribution plot ─────────────────────────────────────
chunk_sizes = [len(c.get('text', '')) for c in kb.chunks]
if chunk_sizes:
    fig, axes = plt.subplots(1, 2, figsize=(12, 4))
    axes[0].hist(chunk_sizes, bins=30, edgecolor='black', alpha=0.7, color='steelblue')
    axes[0].set_xlabel('Chunk Size (characters)')
    axes[0].set_ylabel('Frequency')
    axes[0].set_title('Chunk Size Distribution')
    axes[0].axvline(np.mean(chunk_sizes), color='red', linestyle='--',
                    label=f'Mean: {np.mean(chunk_sizes):.0f}')
    axes[0].legend()
    axes[1].boxplot(chunk_sizes, vert=True)
    axes[1].set_ylabel('Chunk Size (characters)')
    axes[1].set_title('Chunk Size Box Plot')
    axes[1].grid(True, alpha=0.3)
    plt.tight_layout()
    plt.savefig(str(REPORTS_DIR / 'rag_chunk_distribution.png'), dpi=150, bbox_inches='tight')
    plt.show()

print(f"\n✓ Total wall time: {time.time() - t0:.1f}s")

# Restore logging
for _name in ['src.rag', 'src.rag.knowledge_base', 'src.rag.document_chunker',
              'src.rag.embedder', 'src.rag.vector_store', 'src.rag.retriever']:
    logging.getLogger(_name).setLevel(logging.INFO)

Step 1/5: Loading embedding model …


Loading weights: 100%|██████████| 103/103 [00:00<00:00, 1524.57it/s, Materializing param=pooler.dense.weight]                             
BertModel LOAD REPORT from: sentence-transformers/all-MiniLM-L6-v2
Key                     | Status     |  | 
------------------------+------------+--+-
embeddings.position_ids | UNEXPECTED |  | 

Notes:
- UNEXPECTED	:can be ignored when loading from different task/architecture; not ok if you expect identical arch.


  done [3.7s]
Step 2/5: Creating documents …
  303 documents [0.9s]
Step 3/5: Chunking documents …
  303 chunks [0.0s]
Step 4/5: Embedding chunks …


## Section 4: Similarity Search & Retrieval

Test retrieval with the target query: "Find past incidents similar to current sensor deviation pattern"

In [ ]:
# ── Query 1: text query ──────────────────────────────────────────────
query1 = "Find past incidents similar to current sensor deviation pattern with high temperature"
print(f"Query 1: {query1}\n")

results1 = kb.search(query1, top_k=5)
print(f"Retrieved {len(results1)} results:\n")
for i, result in enumerate(results1, 1):
    print(f"Result {i}:")
    print(f"  Score: {result['score']:.3f}")
    print(f"  Text:  {result['text'][:200]}...")
    citation = result.get('citation', {})
    if citation:
        print(f"  Citation: {citation.get('citation_string', 'N/A')}")
    print()

# ── Query 2: degradation pattern ────────────────────────────────────
print("=" * 80)
query2 = "Silent degradation with gradual RUL decrease and multiple anomalies detected"
print(f"\nQuery 2: {query2}\n")

results2 = kb.search(query2, top_k=5)
print(f"Retrieved {len(results2)} results:\n")
for i, result in enumerate(results2, 1):
    meta = result.get('metadata', {})
    print(f"Result {i}:")
    print(f"  Score: {result['score']:.3f}")
    print(f"  Text:  {result['text'][:150]}...")
    if 'engine_id' in meta:
        print(f"  Engine: {meta['engine_id']}")
    if 'duration' in meta:
        print(f"  Duration: {meta['duration']} cycles")
    print()

# ── Query 3: sensor deviation pattern ───────────────────────────────
print("=" * 80)
print("\nQuery 3: Current sensor deviation pattern\n")

sensor_deviations = {
    'sensor_2':  0.45,   # Temperature increase
    'sensor_3': -0.32,   # Pressure decrease
    'sensor_4':  0.28,   # Speed deviation
    'sensor_11': 0.51,   # High deviation
    'sensor_15': -0.19   # Minor decrease
}

results3 = kb.search_similar_failures(
    sensor_deviations=sensor_deviations,
    rul=75.0,
    anomaly_score=0.65,
    top_k=5
)

print(f"Sensor Deviations: {sensor_deviations}")
print(f"RUL: 75.0, Anomaly Score: 0.65\n")
print(f"Retrieved {len(results3)} similar failures:\n")
for i, result in enumerate(results3, 1):
    citation = result.get('citation', {})
    print(f"Similar Failure {i}:")
    print(f"  Similarity: {result['score']:.3f}")
    print(f"  Description: {result['text'][:180]}...")
    if citation:
        print(f"  Citation: {citation.get('citation_string', 'N/A')}")
    print()

# ── Visualise similarity scores ─────────────────────────────────────
fig, axes = plt.subplots(1, 3, figsize=(15, 4))
for idx, (results, qname) in enumerate([
    (results1, 'Query 1'), (results2, 'Query 2'), (results3, 'Query 3')
]):
    scores = [r['score'] for r in results]
    ranks  = list(range(1, len(scores) + 1))
    axes[idx].bar(ranks, scores, color='steelblue', edgecolor='black', alpha=0.7)
    axes[idx].set_xlabel('Result Rank')
    axes[idx].set_ylabel('Similarity Score')
    axes[idx].set_title(f'{qname}\nSimilarity Scores')
    axes[idx].set_xticks(ranks)
    axes[idx].grid(True, alpha=0.3)
    axes[idx].set_ylim([0, 1])

plt.tight_layout()
plt.savefig(str(REPORTS_DIR / 'rag_similarity_scores.png'), dpi=150, bbox_inches='tight')
plt.show()
print("✓ Similarity search completed successfully")

## Section 5: Validate Retrieval Relevance

Manual validation of retrieval quality using test cases.

In [ ]:
# create_test_cases_from_degradation already imported in Cell 3

logger.info("Creating test cases for validation...")
test_cases = create_test_cases_from_degradation(
    degradation_periods,
    n_test_cases=20
)

print(f"Created {len(test_cases)} test cases\n")
print("Example test cases:")
for i, tc in enumerate(test_cases[:3], 1):
    print(f"\n  Test Case {i}:")
    print(f"    Query: {tc['query']}")
    print(f"    Expected: Engine {tc['expected_results']}")

# ── Validate retrieval ──────────────────────────────────────────────
logger.info("Validating retrieval quality...")
validation_metrics = kb.validate(
    test_cases=test_cases,
    query_field='query',
    expected_field='expected_results'
)

print(f"\n{'=' * 60}")
print("RETRIEVAL VALIDATION METRICS")
print(f"{'=' * 60}\n")
print(f"Test Cases:          {validation_metrics['n_test_cases']}")
print(f"Top-1 Accuracy:      {validation_metrics['top_1_accuracy']:.1%}")
print(f"Top-5 Recall:        {validation_metrics['top_5_recall']:.1%}")
print(f"Mean Reciprocal Rank:{validation_metrics['mrr']:.3f}")

# ── Manual inspection ───────────────────────────────────────────────
print(f"\n{'=' * 60}")
print("MANUAL INSPECTION (First 3 Test Cases)")
print(f"{'=' * 60}\n")

for i, tc in enumerate(test_cases[:3], 1):
    query = tc['query']
    expected_engine = tc['expected_results']
    print(f"Test Case {i}:")
    print(f"  Query: {query}")
    print(f"  Expected Engine: {expected_engine}\n  Top 5 Results:")
    results = kb.search(query, top_k=5)
    for rank, r in enumerate(results, 1):
        eid   = r.get('metadata', {}).get('engine_id', 'N/A')
        score = r['score']
        match = "✓" if eid == expected_engine else " "
        print(f"    {rank}. Engine {eid} | Score: {score:.3f} {match}")
    print()

# ── Visualise validation ────────────────────────────────────────────
metrics_names  = ['Top-1 Accuracy', 'Top-5 Recall', 'MRR']
metrics_values = [
    validation_metrics['top_1_accuracy'],
    validation_metrics['top_5_recall'],
    validation_metrics['mrr']
]

fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 5))

colors = ['#2ecc71' if v >= 0.7 else '#e74c3c' for v in metrics_values]
bars = ax1.bar(metrics_names, metrics_values, color=colors, edgecolor='black', alpha=0.7)
ax1.set_ylabel('Score')
ax1.set_title('Retrieval Validation Metrics')
ax1.set_ylim([0, 1])
ax1.axhline(0.7, color='orange', linestyle='--', label='Target (70%)')
ax1.grid(True, alpha=0.3)
ax1.legend()
for bar, value in zip(bars, metrics_values):
    ax1.text(bar.get_x() + bar.get_width() / 2., bar.get_height(),
             f'{value:.1%}', ha='center', va='bottom', fontweight='bold')

# Retrieval quality heatmap
n_test = len(test_cases)
quality_matrix = np.zeros((n_test, 5))
for i, tc in enumerate(test_cases):
    results = kb.search(tc['query'], top_k=5)
    expected = tc['expected_results']
    for rank, r in enumerate(results):
        eid = r.get('metadata', {}).get('engine_id', -1)
        if eid == expected:
            quality_matrix[i, rank] = r['score']

n_show = min(15, n_test)
im = ax2.imshow(quality_matrix[:n_show], aspect='auto', cmap='RdYlGn', vmin=0, vmax=1)
ax2.set_xlabel('Result Rank')
ax2.set_ylabel('Test Case')
ax2.set_title('Retrieval Quality Heatmap\n(Similarity when expected result found)')
ax2.set_xticks(range(5))
ax2.set_xticklabels([f'Rank {i+1}' for i in range(5)])
ax2.set_yticks(range(n_show))
ax2.set_yticklabels([f'TC {i+1}' for i in range(n_show)])
plt.colorbar(im, ax=ax2, label='Similarity Score')

plt.tight_layout()
plt.savefig(str(REPORTS_DIR / 'rag_validation_metrics.png'), dpi=150, bbox_inches='tight')
plt.show()

# Pass / Fail
target_top5 = 0.70
if validation_metrics['top_5_recall'] >= target_top5:
    print(f"\n✓ VALIDATION PASSED: Top-5 recall {validation_metrics['top_5_recall']:.1%} >= {target_top5:.0%}")
else:
    print(f"\n✗ VALIDATION BELOW TARGET: Top-5 recall {validation_metrics['top_5_recall']:.1%} < {target_top5:.0%}")
print("✓ Validation completed")

## Section 6: Citation Tracking & Export

Demonstrate citation tracking and export results for downstream use.

In [ ]:
import csv, json

# ── Retrieve with full citations ────────────────────────────────────
query = "Find past incidents similar to current sensor deviation pattern"
results = kb.search(query, top_k=5)

print("CITATION TRACKING DEMONSTRATION")
print("=" * 80)
print(f"\nQuery: {query}\n")

for i, result in enumerate(results, 1):
    print(f"\nResult {i}:")
    print("-" * 60)
    print(f"Text: {result['text'][:250]}...")
    meta = result.get('metadata', {})
    if meta:
        print(f"\nMetadata:")
        for k, v in meta.items():
            print(f"  {k}: {v}")
    citation = result.get('citation', {})
    if citation:
        print(f"\nCitation Information:")
        for k, v in citation.items():
            if k != 'citation_string':
                print(f"  {k}: {v}")
        print(f"\nFormatted Citation:\n  {citation.get('citation_string', 'N/A')}")
    print()

# ── Export results ──────────────────────────────────────────────────
reports_dir = Path('../reports')
reports_dir.mkdir(parents=True, exist_ok=True)

results_export = []
for result in results:
    citation = result.get('citation', {})
    meta = result.get('metadata', {})
    results_export.append({
        'rank':         result.get('rank', 0),
        'score':        result['score'],
        'text':         result['text'],
        'engine_id':    meta.get('engine_id', None),
        'failure_type': meta.get('failure_type', None),
        'duration':     meta.get('duration', None),
        'citation':     citation.get('citation_string', ''),
        'retrieved_at': citation.get('retrieved_at', '')
    })

csv_path = reports_dir / 'rag_retrieval_results.csv'
if results_export:
    with open(csv_path, 'w', newline='') as f:
        writer = csv.DictWriter(f, fieldnames=results_export[0].keys())
        writer.writeheader()
        writer.writerows(results_export)
    print(f"✓ Results exported to {csv_path}")

json_path = reports_dir / 'rag_retrieval_results.json'
with open(json_path, 'w') as f:
    json.dump(results_export, f, indent=2, default=str)
print(f"✓ Results exported to {json_path}")

# ── Query history analytics ─────────────────────────────────────────
query_history = kb.retriever.get_query_history()
print(f"\n{'=' * 80}")
print("QUERY HISTORY ANALYTICS")
print(f"{'=' * 80}\n")
print(f"Total queries: {len(query_history)}")

if query_history:
    print(f"\nRecent queries:")
    for i, qr in enumerate(query_history[-5:], 1):
        print(f"\n  {i}. Query: {qr['query'][:80]}...")
        print(f"     Results: {qr['n_results']}  |  Top score: {qr['top_score']:.3f}")
        print(f"     Timestamp: {qr['timestamp']}")

# ── Retriever statistics ────────────────────────────────────────────
retriever_stats = kb.retriever.get_statistics()
print(f"\n{'=' * 80}")
print("RETRIEVER STATISTICS")
print(f"{'=' * 80}\n")
for k, v in retriever_stats.items():
    print(f"  {k}: {v:.3f}" if isinstance(v, float) else f"  {k}: {v}")

print(f"\n✓ Citation tracking and export completed")

## Summary

**PHASE 6 RAG Pipeline Complete — All 4 C-MAPSS Subsets (FD001-FD004)**

**Data Coverage:**
- ✓ FD001 (100 engines) + FD002 (260 engines) + FD003 (100 engines) + FD004 (248 engines)
- ✓ Composite engine IDs (`FD001_10`, `FD002_3`, …) prevent cross-subset contamination
- ✓ Feature engineering via `FeatureEngineeringPipeline` (30-cycle windows, 20 selected features)

**Achievements:**
- ✓ Built knowledge base from combined degradation data (all 4 subsets)
- ✓ Document chunking with sentence-based strategy
- ✓ Generated embeddings using sentence-transformers (`all-MiniLM-L6-v2`, 384-dim)
- ✓ Created FAISS vector store for efficient similarity search
- ✓ Implemented retrieval with citation tracking
- ✓ Validated retrieval relevance (Top-5 recall target: ≥ 70 %)
- ✓ Exported results to CSV/JSON under `reports/`

**Key Metrics:**
- Documents & chunks covering all subsets
- Embedding dimension: 384
- Retrieval validation: Top-1 accuracy, Top-5 recall, MRR
- Citation tracking for every retrieved result

**Next Steps:**
- NB06: Agentic Architecture — integrate RAG retriever with reasoning agents
- Optional: cross-encoder re-ranking, LLM-based natural language generation